<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-and-Prompt-Engineering/blob/main/VertexAI_Gemini_Prompt_Engineering_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Techniques

---




### Install Vertex AI SDK for Python


In [1]:
! pip3 install --upgrade --user --quiet google-cloud-aiplatform

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.


In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
PROJECT_ID = "jrproject-402905"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_ID = "gemini-1.5-flash-preview-0514"  # @param {type:"string"}
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
# load the model
model = GenerativeModel(MODEL_ID, system_instruction=[ "You are a helpful assistant.","Your answer questions in a concise way",],)

# Set model parameters
generation_config = GenerationConfig( temperature=0.9, top_k=32,)

def generate_response(prompt,model=model):
  contents = [prompt]
  response = model.generate_content(contents, generation_config=generation_config,)
  return response.text

We will use Gemini 1.5 Flash model for these given below prompting techniques

### There are different prompting techniques:
**1) Multi-turn conversation or Chatbot with conversation history** <br>
**2) Zero-shot Prompting** <br>
**3) Few-shot Prompting** <br>
**4) Chain-of-Thought Prompting** <br>
**5) Tree-of-Thought Prompting** <br>
**6) ReAct Prompting(Reason and Act)** <br>

**Zero-shot prompting**

Zero-shot prompting means that the prompt used to interact with the model won't contain examples or demonstrations. The zero-shot prompt directly instructs the model to perform a task without any additional examples to steer it.

In [5]:
prompt='''
Message: Hi alex, thanks for the thoughtful birthday card!
Sentiment: ?
'''

ans=generate_response(prompt)

print(ans)


Appreciation and gratitude. 



**Few-shot prompting**

Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

In [6]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?
"""

ans=generate_response(prompt)

print(ans)

Positive 



### Specifying the Output Format
- You can also specify the format in which you want the model to respond.
- In the example below, you are asking to "give a one word response".

In [7]:
prompt = """
Message: Hi Dad, you're 20 minutes late to my piano recital!
Sentiment: Negative

Message: Can't wait to order pizza for dinner tonight
Sentiment: Positive

Message: Hi Amit, thanks for the thoughtful birthday card!
Sentiment: ?

Give a one word response.
"""

ans=generate_response(prompt)

print(ans)

Positive 



**Chain-of-Thought Prompting**
chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

In [8]:
prompt = """
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""

ans=generate_response(prompt)

print(ans)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is **False**. 



**Tree of Thoughts (ToT)**

Tree of Thoughts (ToT) prompting is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem-solving with language models.


How does it work?

- ToT prompting breaks problems down into smaller parts, similar to CoT prompting, but goes further by combining this with the ability to explore multiple solution paths in parallel, forming a tree instead of a single chain. Each thought is generated or solved independently and passed to the next step, allowing the model to self-evaluate and decide whether to continue with that path or choose another.

In [9]:

prompt = """
Imagine 5 different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on science and the laws of physics
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought
Each expert will assign a likelihood of their current assertion being correct
Continue until the experts agree on the single most likely location
The question is.
1. Carlos is at the swimming pool.
2. He walks to the locker room, carrying a towel.
3. He puts his watch in the towel and carries the towel tightly to a lounger at the poolside.
4. At the lounger he opens and vigorously shakes the towel, then walks to the snack bar.
5. He leaves the towel at the snack bar, then walks to the diving board.
6. Later Carlos realises he has has lost his watch. Where is the single most likely location of the watch?
"""


ans=generate_response(prompt)

print(ans)

## Expert Brainstorming:

**Expert 1:**  The watch was last seen in the towel. **Likelihood: 100%**

**Expert 2:** The towel was shaken vigorously, so the watch likely fell out. **Likelihood: 90%**

**Expert 3:** The towel was left at the snack bar. **Likelihood: 100%**

**Expert 4:** Therefore, the watch is most likely at the snack bar, where the towel was last seen. **Likelihood: 95%**

**Expert 5:**  However, Carlos walked to the diving board after leaving the towel.  It's possible he dropped the watch while walking. **Likelihood: 70%**

**Expert 1:**  Good point. The watch could have fallen out while walking to the diving board. But the towel was shaken vigorously, making it more likely to have fallen out there. **Likelihood: 80%**

**Expert 2:** We need to consider the laws of physics. A watch is relatively heavy and wouldn't easily fly out of a tightly held towel. It's more probable that it fell out during the shake.  **Likelihood: 95%**

**Expert 3:** Agreed. The vigorous shakin

**ReAct Prompting**

ReAct is a framework where LLMs are used to generate both reasoning traces and task-specific actions in an interleaved manner.

Generating reasoning traces allow the model to induce, track, and update action plans, and even handle exceptions. The action step allows to interface with and gather information from external sources such as knowledge bases or environments.

In [23]:
prompt = """
Solve a question answering task with interleaving
Thought,
Action,
Observation steps.

Thought can reason about the current situation, and Action can be three types:

Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.

Calculator[query], it performs mathematical calculations based on input query.

Lookup[keyword], which returns the next sentence containing keyword in the current passage.

Finish[answer], which returns the answer and finishes the task.

Question:

Who is Elon Musk?
"""


ans=generate_response(prompt)

print(ans)

Thought: I need to find information about Elon Musk.
Action: Search[Elon Musk]
Observation: Elon Reeve Musk (born June 28, 1971) is a business magnate and entrepreneur. He is the founder, CEO and Chief Engineer of SpaceX; early investor, CEO and Product Architect of Tesla, Inc.; founder of The Boring Company; and co-founder of Neuralink and OpenAI. 
Thought: I have the information about Elon Musk. 
Action: Finish[Elon Musk is a business magnate and entrepreneur. He is the founder, CEO and Chief Engineer of SpaceX; early investor, CEO and Product Architect of Tesla, Inc.; founder of The Boring Company; and co-founder of Neuralink and OpenAI.] 



**2. Chain-of-Verification Prompting**

The Chain-of-Verification (CoVe) prompt engineering method aims to reduce hallucinations through a verification loop.

In [21]:
Question="Name some athletes who were born in United states"

prompt=f'''Here is the question: {Question}.

First, generate a response.

Then, create and answer verification questions based on this response to check for accuracy. Think it through and make sure you are extremely accurate based on the question asked.

After answering each verification question, consider these answers and revise the initial response to formulate a final, verified answer. Ensure the final response reflects the accuracy and findings from the verification process.'''



ans=generate_response(prompt)

print(ans)


## Initial Response:

Some athletes born in the United States are Michael Jordan, Serena Williams, Tom Brady, and LeBron James.

## Verification Questions:

1. **Is Michael Jordan a U.S. citizen?** 
    * **Answer:** Yes.
2. **Is Serena Williams a U.S. citizen?** 
    * **Answer:** Yes.
3. **Is Tom Brady a U.S. citizen?** 
    * **Answer:** Yes.
4. **Is LeBron James a U.S. citizen?** 
    * **Answer:** Yes.

## Revised Response:

The following athletes were born in the United States: Michael Jordan, Serena Williams, Tom Brady, and LeBron James. 



**3. Step-Back Prompting**

A crucial rule to remember when prompting LLMs is to give them space to 'think'. You don’t want to overly constrain the model such that it can’t explore various solutions.

Chain of thought reasoning is one way to push the model to think through the problem. A simple way to implement this type of reasoning is to add a statement like “think through this task step-by-step” at the end of your prompt.

In [22]:
Question="What is Quantum Physics?"
prompt=f'''Here is a question or task: {Question}

Let's think step-by-step to answer this:

Step 1) Abstract the key concepts and principles relevant to this question:

Step 2) Use the abstractions to reason through the question:

Final Answer: '''


ans=generate_response(prompt)

print(ans)



## Step 1: Key Concepts and Principles

* **Quantum Mechanics:** A fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles. 
* **Quantization:** The idea that certain physical quantities, like energy, can only exist in discrete values, rather than continuous ones.
* **Wave-particle duality:** The concept that particles can exhibit wave-like properties and waves can exhibit particle-like properties.
* **Uncertainty Principle:**  States that certain pairs of physical properties, like position and momentum, cannot be known with perfect accuracy simultaneously.
* **Superposition:**  A quantum state where a system exists in multiple possible states at the same time until measured.
* **Entanglement:** A phenomenon where two or more particles become linked, even if separated by vast distances, such that measuring the state of one instantaneously affects the state of the other.

## Step 2: Reasoning

Quantum physics deals wit

## Thank You